# Training example, the 2018 Mw7.5 Palu, Sulawesi earthquake

This is a coarse version of the complex 3D dynamic rupture model published in [Ulrich et al., 2019](https://link.springer.com/article/10.1007/s00024-019-02290-5).

The published high-resolution model setup is available at [zenodo](https://zenodo.org/record/3234664#.YVFmIi8Ro6h). 

This earthquake scenario, featuring sustained supershear rupture propagation, matches key earthquake observational data, including its moment magnitude, rupture duration, fault plane solution, teleseismic waveforms and inferred horizontal ground displacements. The applied remote stress regime reflecting regional transtension, produces a combination of up to 6 m left-lateral slip and up to 2 m normal slip on the straight fault segment dipping 65 degree East beneath Palu Bay. We will model left-lateral, dominantly strike-slip faulting on bend, connected fault segments. 
![](Ulrich19_Fig1.png)
*Tectonic setting of the September 28, 2018 Mw 7.5 Sulawesi
earthquake (epicenter indicated by yellow star). Black lines indicate plate boundaries based on Bird (2003); Socquet et al. (2006); Argus et al. (2011). BH Bird’s Head plate, BS Banda Sea plate, MF Matano fault zone, PKF Palu-Koro fault zone, MS Molucca Sea plate, SSF Sula-Sorong fault zone, TI Timor plate. Arrows indicate the far-field plate velocities with respect to Eurasia (Socquet et al. 2006). The black box corresponds to the region displayed in b. b A zoom of the region of interest. The site of the harbor tide gauge of Pantoloan is indicated as well as the city of Palu. Locations of the GPS stations at which we provide synthetic ground displacement time series (see Appendix 7.2) are indicated by the red triangles. Focal mechanisms and epicenters of the September 28, 2018 Palu earthquake (USGS (2018a), top), October 1, 2018 Palu aftershock (middle), and January 23, 2005 Sulawesi earthquake (bottom) are shown. These later two events provide constraints on the dip angles of individual segments of the fault network. Individual fault segments of the Palu-Koro fault used in the dynamic rupture model are coloured. c, d, e 3D model of the fault network viewed from top, SW and S*


## Mesh

To build 3D models with complicated geometries beyond gmsh's capabilties we are using [SimModeler](http://www.simmetrix.com/index.php/simmodeler/overview), a software that is free for acaedmic users. 
The neccesary steps include:
1. Creating a high resolution topography and bathymetry free surface and merge it with a simple 3D volume (e.g. a box);
2. Creating the complex fault network constrained by fault traces and varying fault dip and intersect with topo-bathymetry;
3. Automatic volumetric meshing using unstructured tetrahedral elements. Keep in mind resolution criteria: the CFL criterion for wave propagation, and the dynamic rutpure process zone, the region behind the rupture front where the fault strength drops from its static to dynamic level (Day et al., 2005; for SeisSol: [Wollherr et al., 2018](https://academic.oup.com/gji/article/214/3/1556/5017447?login=true)).

We recorded two non-narrated demos of these steps for the Palu, Sulawesi model, which are available [here](https://drive.google.com/file/d/1Y3mTAoPTAyUMdXfTzW_Ap62u4EpJM0Zf/view?usp=sharing) and [here](https://drive.google.com/file/d/1ts3QZCWUeHMwB3ZEhUpV-CmNcG-4Vwj8/view?usp=sharing).

For the sake of time we here provide a coarse mesh file and merely visualize it. The prescribed resolution here is 800 m element edge length everywhere on the fault, and rapid static coarsening away from the fault system. This mesh has ~215k elements. The high-resolution mesh for comparison consists of 8 million elements, decreasing the shortest element edge lengths to 200 m close to faults.

In [ ]:
import vtk
import pyvista as pv

reader = vtk.vtkXdmfReader()
reader.SetFileName('Sulawesi_65dip_straightBay_ShortNorth_micro.xdmf')
reader.Update()
mesh = pv.wrap(reader.GetOutput())
elevation = mesh.elevation(low_point=(0, 0, -800), high_point=(0, 0, 2970))
pv.plot(elevation, cmap='terrain', background='grey', show_edges=False, jupyter_backend='static')

## Run the code

In the following we start SeisSol using the provided parameters file. Note, that we combine two 3D heterogeneous subsurfave velocity data sets using [ASAGI](https://seissol.readthedocs.io/en/latest/asagi.html), our open source software for effecient reading, interpolation and writing of parallel, adaptive geoinformation ([Rettenberger et al., 2016](http://dl.acm.org/citation.cfm?id=2938618)).

This is how the high-resolution rupture looks like:
![](Ulrich19_Fig3.png)
*a) Snapshot of the wavefield (absolute particle velocity in m/s) and the slip rate (in m/s) across the fault network at a rupture time of 15 s. b) Overview of the simulated rupture propagation. Snapshots of the absolute slip rate are shown at a rupture time of 2, 9, 13, 23 and 28 s. Labels indicate noteworthy features of the rupture*



You should adjust the OMP_NUM_THREADS variable to match the number of cores on your PC. This simulation will run for some time. In case of trouble try allowing more RAM (ideally 5-8 GB) in the settings of docker.

Note: this model, including fast velocity weakening friction, will run faster if compiled with a commercial compiler such as intel. as available on clusters. We believe SeisSol's current rate & state implementation is slow when compiled with the gnu compiler, and suspect that the auto-vectorization of the math functions does not work very well with gcc.


In [ ]:
!OMP_NUM_THREADS=4 SeisSol_Release_dhsw_4_elastic parameters.par

## Visualization

We now visualize the fault output generated by SeisSol. This visualises all processes occuring across the fault surface during dynamic rupture, e.g. slip rates, slip, shear and normal stresses, effective friction etc.

Check out the [documentation](https://seissol.readthedocs.io/en/latest/fault-output.html#outputmask) for an explanation of the variable names.

In [ ]:
import vtk
import pyvista as pv
from ipywidgets import interact

reader = vtk.vtkXdmfReader()
reader.SetFileName('output/Sulawesi-fault.xdmf')
reader.Update()
cd = reader.GetOutput().GetCellData()
variables = [cd.GetArrayName(i) for i in range(cd.GetNumberOfArrays())]

@interact(t=(0.0, 30.0, 0.5), var=variables)
def plot(t=0.0, var='SRs'):
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('grey')
    plotter.add_mesh(mesh, cmap='Blues', scalars=var)
    plotter.show(jupyter_backend='static')

In the following we visualize the free surface output.
Fields *u, v, w* are the particle velocities and *U, V, W* are the displacements in *x, y, z* direction, respectively.

In [ ]:
reader = vtk.vtkXdmfReader()
reader.SetFileName('output/Sulawesi-surface.xdmf')
reader.Update()
cd = reader.GetOutput().GetCellData()
variables = [cd.GetArrayName(i) for i in range(cd.GetNumberOfArrays())]

@interact(t=(0.0, 30.0, 0.25), var=variables)
def plot(t=0.0, var='W'):
    reader.UpdateTimeStep(t)
    mesh = pv.wrap(reader.GetOutput())
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('grey')
    plotter.add_mesh(mesh, cmap='Blues', scalars=var)
    plotter.show(jupyter_backend='static')